<a href="https://colab.research.google.com/github/aviramberg276/personality/blob/develop/use_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
# import hypertools as hyp
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf

sb.set(style="darkgrid")
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
database = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/personality/Aviram_dataset_wide_A.csv", index_col=[0])

In [0]:
classes = database[database.columns[1:5]]
data = database[database.columns[5:]]

In [0]:
import keras.backend as K
def custom_loss(layer):

    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
        return K.mean(K.mean(y_true, axis=0), axis=-1)
    return loss

val = 0
def loss3(y_true,y_pred):
    global val
    val= K.sum(K.sqrt(y_true -y_pred))
    # print(K.sum((y_true -y_pred)**2))
    return K.sum((y_true -y_pred)**2)

def loss2(y_true,y_pred, num):
#         return K.mean(K.square(y_pred - y_true) + K.square(layer), axis=-1)
    return (np.abs((y_true - y_test)) < num).sum() / (y_test.shape[0] * y_test.shape[1]) * 100

def mapping_to_target_range( x, target_min=1, target_max=7) :
    x02 = K.tanh(x) + 1 # x in range(0,2)
    scale = ( target_max-target_min )/2.
    return  x02 * scale + target_min

In [0]:
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import optimizers
seed =7
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(1500, input_dim=2910, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.15, seed=np.random.seed(seed)))
    model.add(Dense(500, input_dim=1000, kernel_initializer='normal'))
    model.add(Dense(200, input_dim=500, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.15, seed=np.random.seed(seed)))
    model.add(Dense(100, input_dim=160, kernel_initializer='normal'))
    model.add(Dense(40, input_dim=100, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.15, seed=np.random.seed(seed)))
    model.add(Dense(4, kernel_initializer='normal', activation=mapping_to_target_range))
    # Compile model
    # optimizer = optimizers.Adam(lr=0.0001)
    optimizer = optimizers.Adam(lr=0.0001)
    model.compile(loss=loss3, optimizer=optimizer, metrics=['accuracy'])
    return model

In [0]:
model = baseline_model()

In [0]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/model/personality/ep5481-loss4.166-accuracy0.902.h5")

In [0]:
from sklearn.model_selection import GroupKFold
group_kfold = GroupKFold(n_splits=15)
group_kfold.get_n_splits(data, classes)
gkf = list(group_kfold.split(data, classes, database[database.columns[0]]))

In [0]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
train_index, test_index = gkf[0]
X_train, X_test = scalar.fit_transform(data.values[train_index]), scalar.fit_transform(data.values[test_index])
y_train, y_test = classes.values[train_index], classes.values[test_index]

In [0]:
model.predict(X_test)[1], y_test[1]

(array([4.2052603, 4.29218  , 4.2697806, 3.979812 ], dtype=float32),
 array([4.16666667, 4.25      , 4.25      , 4.        ]))

In [0]:
model.predict(X_test)[10], y_test[10]

(array([4.1500783, 4.4199066, 5.010091 , 2.8339734], dtype=float32),
 array([4.08333333, 4.5       , 5.        , 2.75      ]))

In [0]:
model.predict(X_test)[9], y_test[9]

(array([6.136693 , 6.2651606, 6.062276 , 6.2053432], dtype=float32),
 array([6.16666667, 6.25      , 6.        , 6.25      ]))

In [0]:
model.predict(X_test)[14], y_test[14]

(array([4.592772 , 4.9981585, 4.974079 , 3.7710307], dtype=float32),
 array([4.58333333, 5.        , 5.        , 3.75      ]))